<a href="https://colab.research.google.com/github/SurinSeong/FinalPJT/blob/main/langchain_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# langchain-basic
[위키독스-랭체인(LangChain) 입문부터 응용까지](https://wikidocs.net/book/14473)

## 환경 구성

In [1]:
## 1. 라이브러리 설치
!pip install -q langchain langchain-openai tiktoken # tiktoken : openai에서 사용하는 tokenizer / -q : 설치 중간 과정을 보고 싶지 않을 때 작성

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 581.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.6/990.6 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.0/384.0 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.2/140.2 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.7/360.7 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.5 MB/s eta 0:00:00


In [4]:
## 2. openai 인증키 설정
import os

os.environ['OPENAI_API_KEY'] = 'FINAL_TEAM2'

In [5]:
# Openai를 만들 수 있는 클래스 가져오기
from langchain_openai import ChatOpenAI

# model 생성
llm = ChatOpenAI(model='gpt-3.5-turbo-0125')

# model에게 prompt 전달
llm.invoke('지구의 자전 주기는?') # AIMessage : 모델의 답변

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: FINAL_TEAM2. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [ ]:
from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# prompt + model + output parser
prompt = ChatPromptTemplate.from_template('You are an expert in astronomy. Answer the question. <Question>: {input}')
llm = ChatOpenAI(model='gpt-3.5-turbo-0125')
output_parser = StrOutputParser() # AIMessage의 content만 반환

# LCEL chaining
chain = prompt | llm | output_parser

# chain 호출
chain.invoke({'input':'지구의 자전 주기는?'})

### 멀티 체인
* 여러 개의 체인을 연결하거나 복합적으로

In [ ]:
# 첫 번째 체인을 위한 prompt
prompt1 = ChatPromptTemplate.from_template('translates {korean_word} to English.')
# 두 번째 체인을 위한 prompt
prompt2 = ChatPromptTemplate.from_template(
    'explain {english_word} using oxford dictionary to me in Korean.'
)

llm = ChatOpenAI(model='gpt-3.5-turbo-0125')

chain1 = prompt1 | llm | StrOutputParser()

chain1.invoke({'korean_word':'미래'})

In [ ]:
chain2 = (
    {'english_word':chain1}
    | prompt2
    | llm
    | StrOutputParser()
)

chain2.invoke({'korean_word':'미래'}) # chain1을 통과하는 것이 먼저 이기 때문에 korena_word를 넣어야 한다.

### PromptTemplate
* 단일문장입력 --> 단일 문장 출력

In [ ]:
from langchain_core.prompts import PromptTemplate

# 'name'과 'age'라는 두 개의 변수를 사용하는 프롬프트 템플릿을 정의한다.
template_text = '안녕하세요. 제 이름은 {name}이고, 나이는 {age}살 입니다.'

# PromptTemplate 인스턴스를 생성한다.
prompt_template = PromptTemplate.from_template(template_text)

# 템플릿에 값을 채워서 프롬프트를 완성한다.
filled_prompt = prompt_template.format(name='홍길동', age=30)

filled_prompt

In [ ]:
# 문자열 템플릿 결합 (+ 연산자 이용 가능)
combined_prompt = (
    prompt_template
    + PromptTemplate.from_template('\n\n아버지를 아버지라 부를 수 없습니다.')
    + '\n\n{language}로 번역해주세요.'
)

combined_prompt

In [ ]:
combined_prompt.format(name='홍길동', age=30, language='영어')

### ChatPromptTemplate
* message 여러 개가 리스트 형태로 배열을 이룬다.
* 여러 의미를 가진 메세지를 받는다.
* 챗봇형태의 모델을 구현할 때 자주 사용함.